This example shows how to use hydrolinking methods to address a multiple points to the NHD High Resolution.


##### First import packages and set functions allowing for import of data via CSV

get_user_input() requests field names from user and standardizes these field names
import_file() uses user input to load file with point data

Note we use a pandas dataframe to help manage and loop through data but this can be done using many other methods.

In [1]:
#Import packages needed for code to run
from hydrolink import nhd_hr
import warnings; warnings.simplefilter('ignore')
import geopandas as gpd
import pandas as pd

def get_user_input():
    #User inputs variables needed for code to run
    input_file = input("Enter file name, including extension (only accepts .csv and .shp): ")
    latitude_field = input("Enter field name for latitude, note this is case sensitive: ")
    longitude_field = input("Enter field name for longitude, note this is case sensitive: ")
    stream_name_field = input("Enter field name for stream name, if none type None, note this is case sensitive: ")
    identifier_field = input("Enter field name for identifier, note this is case sensitive: ")
    crs = input("Enter crs number, recommended to use NAD83 represented by 4269: ")
    in_data = {"file":input_file,"lat":latitude_field,"lon":longitude_field,"stream_name":stream_name_field,"id":identifier_field,"crs":crs}
    return in_data


def import_file(in_data):
    '''
    Description: Imports CSV or SHP files into a Pandas dataframe. Makes sure that all fields 
    are in the file and converts the file into a dataframe with a standard set of field names.
    
    Input
    ------------
    in_data: dictionary with values for file, lat, lon, stream_name, id
    
    Output
    ------------
    Pandas Dataframe
    '''
    #Check to see if the file is a CSV file
    if in_data['file'].endswith('.csv'):
        print ('reading csv file' +'\n')
        try:
            df = pd.read_csv(in_data['file'],encoding='iso-8859-1')
        except KeyError:
            print ('file did not properly import, verify file name and rerun')
            #It would be nice here to reask for inputFileName and then restart at try statement
    
    #If input file is not a CSV check to see if it is a shapefile
    elif in_data['file'].endswith('.shp'):
        print('\n' + 'reading shapefile' + '\n')
        try:
            df = gpd.GeoDataFrame.from_file(in_data['file'])
        except KeyError:
            print ('file did not properly import, verify file name and rerun')
            #It would be nice here to reask for inputFileName and then restart at try statement
    
    #If input file is not a CSV or shapefile tell the user that the file type is not excepted
    else:
        print('File type not currently accepted. Please try .csv or .shp')
    
    

    if in_data['lat'] in df and in_data['lon'] in df and in_data['id'] in df:
        if in_data['stream_name'] is not None and in_data['stream_name'] in df:
            df = df[[in_data['id'], in_data['lat'], in_data['lon'], in_data['stream_name']]].copy()
            df = df.rename(columns={in_data['id']: 'id', in_data['lat']: 'lat', in_data['lon']: 'lon', in_data['stream_name']: 'stream'})
            return df
        else:
            df = df[[in_data['id'], in_data['lat'], in_data['lon']]].copy()
            df = df.rename(columns={in_data['id']: 'id', in_data['lat']: 'lat', in_data['lon']: 'lon'})
            df['stream'] = None
            return df
    else: 
        print ('verify field names and rerun')

In [2]:
input_data = get_user_input()

Enter file name, including extension (only accepts .csv and .shp): test-data/test.csv
Enter field name for latitude, note this is case sensitive: y
Enter field name for longitude, note this is case sensitive: x
Enter field name for stream name, if none type None, note this is case sensitive: stream
Enter field name for identifier, note this is case sensitive: id
Enter crs number, recommended to use NAD83 represented by 4269: 4269


##### Next loop through each point, hydrolinking each

In [3]:
df = import_file(input_data)   
input_crs= 4269
buffer_m= 1000

for row in df.itertuples():
    #Define variables based on row,field values
    input_lat = float(row.lat)
    input_lon = float(row.lon)
    input_identifier = row.id
    stream_name = str(row.stream)

    #initiate a Python object
    hydrolink = nhd_hr.HighResPoint(input_identifier, input_lat, input_lon, water_name=stream_name, input_crs=input_crs, buffer_m=buffer_m)

    #builds query against USGS hem web services
    hydrolink.build_nhd_query(service='hem_flow')

    #executes query and measures distances from point to each line and their nodes
    hydrolink.find_closest_reaches(n=6)

    #stream name match for each of the reaches
    hydrolink.water_name_match()
    
    #Return reach most likely associated with point
    hydrolink.select_best_reach()

    #get address of hydrolinked point
    hydrolink.get_hl_measure()

    #Creates a CSV file of all reach options.  This can be used for QAQC procedures.
    hydrolink.write_reach_options(outfile_name='test-data/test_hr_hydrolink_reach_output.csv')

    #Creates a CSV file with one reach per point.  
    hydrolink.write_best(outfile_name='test-data/test_hr_hydrolink_output.csv')

reading csv file

